In [1]:
import os
import torch
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

2
NVIDIA RTX A5000


In [2]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, PreTrainedModel, EsmConfig, EsmPreTrainedModel
from peft import PeftModel, LoraConfig, get_peft_model, TaskType
# from tomlkit import value
import torch
import esm
import pandas as pd
import numpy as np
import random
import os
import wandb
import pickle as pkl
from datetime import datetime
import accelerate
from accelerate import Accelerator
from huggingface_hub import notebook_login
from torch.utils.data import Dataset, random_split
from transformers import (
    EsmForTokenClassification,
    EsmForMaskedLM,
    EsmModel,
    EsmTokenizer,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

In [3]:
concat_all_exp_data = pd.read_pickle('/home/kaustubh/RuBisCO_ML/ESM_LoRA/data/processed_combined_all_exp_assays_data.pkl')

In [4]:
formIII_lsu_variant_data_df = concat_all_exp_data.query('LSU_id.str.startswith("Anc393") or LSU_id.str.startswith("Anc367") or LSU_id == "Anc365" or LSU_id == "Anc366"')
formIII_lsu_variant_data_df['fixed_threshold_activity'] = formIII_lsu_variant_data_df['mean_reading'].apply(lambda x: 1 if x >= 50 else 0)

formIII_lsu_variant_data_df[formIII_lsu_variant_data_df['activity_binary'] == 1].shape[0], formIII_lsu_variant_data_df[formIII_lsu_variant_data_df['activity_binary'] == 0].shape[0]
# formIII_lsu_variant_data_df[formIII_lsu_variant_data_df['fixed_threshold_activity'] == 1].shape[0], formIII_lsu_variant_data_df[formIII_lsu_variant_data_df['fixed_threshold_activity'] == 0].shape[0]

/tmp/ipykernel_3133278/223130225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formIII_lsu_variant_data_df['fixed_threshold_activity'] = formIII_lsu_variant_data_df['mean_reading'].apply(lambda x: 1 if x >= 50 else 0)


(44, 60)

In [5]:
sequences = formIII_lsu_variant_data_df['lsussu_seq'].to_list()
binary_activity = formIII_lsu_variant_data_df['fixed_threshold_activity'].to_list()

In [6]:
accelerator = Accelerator()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [86]:
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")

In [87]:
class ProteinDataset(Dataset):
    def __init__(self, sequences, binary_activity, tokenizer, max_length=512):
        self.sequences = sequences
        self.binary_activity = binary_activity
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx][:self.max_length]
        binding_site = self.binary_activity[idx]
        encoding = self.tokenizer(sequence, truncation=True, padding='max_length', max_length=self.max_length)
        encoding['labels'] = binding_site # + [-100] * (self.max_length - len(binding_site))  # Ignore extra padding tokens
        return encoding

The `ProteinDataset` class is a custom dataset class that inherits from `torch.utils.data.Dataset`. It is designed to handle protein sequences and their corresponding binding sites for use in a machine learning model. Here's a breakdown of what each part of the class does:

- **`__init__` method**: This initializes the dataset with sequences, binding sites, a tokenizer, and an optional maximum length for the sequences.
    - `sequences`: A list of protein sequences.
    - `binary_activity`: A list of active-inactive labels corresponding to each sequence.
    - `tokenizer`: A tokenizer to convert sequences into token IDs.
    - `max_length`: The maximum length of the sequences to be tokenized (default is 512).

- **`__len__` method**: This returns the number of sequences in the dataset.

- **`__getitem__` method**: This retrieves a single item (sequence and its binding site) from the dataset at the specified index (`idx`).
    - It truncates the sequence to the maximum length.
    - It tokenizes the sequence using the provided tokenizer.
    - It adds the binding site label to the tokenized output.

The class is used to prepare the data for training and evaluation in a format that can be easily fed into a machine learning model.

In [88]:
dataset = ProteinDataset(sequences, binary_activity, tokenizer)
train_size = int(0.85 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))
train_dataset, val_dataset = accelerator.prepare(train_dataset, val_dataset)

In [89]:
# Extract back the sequences and their binding sites info for the entries in the train_dataset & val_dataset
train_sequences = [sequences[i] for i in train_dataset.indices]
train_binary_activity = [binary_activity[i] for i in train_dataset.indices]

val_sequences = [sequences[i] for i in val_dataset.indices]
val_binary_activity = [binary_activity[i] for i in val_dataset.indices]

In [90]:
train_sequences, train_binary_activity

(['...'],
 [0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0])

In [91]:
val_sequences, val_binary_activity

(['MSKRYTDYVDLNYTPKENDLICTFHIEPADGVDLEEAAGRVAAESSIGTWTDVSTMPEIVEKLKARVYEIDESGNIVKIAYPLDLFEPGNIPQILSSIAGNIFGMKAVKNLRLLDIRFPKELVKSFKGPKFGIEGVRELLGVYDRPLVGTIVKPKVGLSAEEHAEVAYEAWVGGLDLVKDDENLTSQPFNPFEERVKKVLEARDKAEEETGEKKVYLVNITAPTEEMIRRAELVADLGGKYVMIDIITAGFSAVQSLREEDLGLAIHAHRAMHAAFTRNPKHGISMLVLAKLARLVGVDQLHIGTVVGKMEGDKEEVLAIRDALRLDRVPADEANHFLEQDWYNIKPVFPVASGGLHPGLVPDLIDIFGKDIIIQAGGGVHGHPDGTRAGAKALRQAIEAAMEGISLEEYAKEHKELKKALEKWGHVR',
  'MSIRYEDFLDLNYEPGDNDLICTFRIEPADGISMEAAASRVASESSNGTWTTLQVMPDRIKKLSATVFEIDGNIVKIAYPADLFEPGNMPQILSSIAGNIMGMKAVDTIRLLDCHWPESLVSSFPGPQFGSSVRRELFGVHDRPLTATVPKPKVGLSAEQHAQIAYEAWVGGLDLIKDDENLTDQPFNPFEERVKKVLAARDKAEEETGEKKAYLVNITAETNEMLERADLVADYGGEYVMIDVITAGWSAVQTLRERCEDLGLAIHAHRAMHAAFTRLPSHGVSMRVLAQIARLVGVDQLHTGTGLGKMEGDEDVLGIADWLRQDLYNINDVFPVASGGLHPGLVPELIEAFGTDICIQAGGGVHGHPDGTRAGAKALRQAVEAAMEGVSLEEYADDHPELATALDKWGTERPR',
  'MSERYEDFVDLSYTPGENDLVCTFRIEPAEGMSMEEAASRVASESSNGTWATLSTMEDIKDLKAKTFSIDGNIIKIAYPLGLFEAGNMPQILSCIAGNIMGMKAVDTLRLLDIHWPEELLSSFKGPQFGSEGRQEIFGVHDR

In [10]:
def model_init(trial):
    base_model = EsmForTokenClassification.from_pretrained(
        "facebook/esm2_t6_8M_UR50D",
        num_labels=2 # For binary classification
    )

    config = LoraConfig(
        task_type=TaskType.TOKEN_CLS,
        r=16,
        lora_alpha=16,
        target_modules=["query", "key", "value"],
        lora_dropout=0.1,
        bias="all",
    )
    
    lora_model = get_peft_model(base_model, config)
    return accelerator.prepare(lora_model)

# config = LoraConfig(
#     r=8,  # Rank of low-rank matrix (controls adaptation strength)
#     lora_alpha=32,  # Scaling factor
#     lora_dropout=0.1,  # Dropout rate for LoRa layers
#     bias="none",
#     target_modules=["self_attn.k_proj", "self_attn.v_proj"],
#     task_type="FEATURE_EXTRACTION"  # Since ESM-2 is a feature extractor
# )

In [11]:
def wandb_hp_space(trial):
    return {
        "method": "random",
        "metric": {"name": "accuracy", "goal": "maximize"},
        "parameters": {
            "learning_rate": {"distribution": "uniform", "min": 1e-5, "max": 1e-3},
            "per_device_train_batch_size": {"values": [2, 4, 8, 16]},
        },
    }

In [12]:
class CustomModel(EsmPreTrainedModel):
    def __init__(self):
        super().__init__(EsmConfig.from_pretrained("facebook/esm2_t6_8M_UR50D"))
        self.backbone = EsmModel.from_pretrained("facebook/esm2_t6_8M_UR50D")

        self.outputs = torch.nn.Linear(320, 1)

    def forward(
        self,
        input_ids,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        labels=None,
        inputs_embeds=None,
        output_attentions=False,
        output_hidden_states=False,
        return_dict=True
    ):
        outputs = self.backbone(
            input_ids,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict
        )

        sequence_output = outputs.last_hidden_state # (B, L, 1280)
        bos_emb = sequence_output[:,0] # (B, L, 1280) -> (B, 1280)
        # outputs = [self.outputs[i](sequence_output) for i in range(5)]
        outputs = self.outputs(bos_emb).squeeze(1) # (B,)

        # if labels, then we are training
        loss = None
        if labels is not None:
            assert outputs.shape == labels.shape, f"{outputs}, {labels}"
            loss_fn = torch.nn.BCEWithLogitsLoss(reduction="mean")
            loss = loss_fn(outputs, labels.float())
            # loss = sum(losses)/len(losses)

        return {
            "loss": loss,
            "logits": outputs
        }
    
    def get_embedding(
        self,
        input_ids,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        labels=None,
        inputs_embeds=None,
        output_attentions=False,
        output_hidden_states=False,
        return_dict=True
    ):
        outputs = self.backbone(
            input_ids,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict
        )

        return outputs

In [13]:
def model_init():
    base_model = CustomModel()
    config = LoraConfig(
        task_type=TaskType.TOKEN_CLS,
        r=16,
        lora_alpha=16,
        target_modules=["query", "key", "value"],  # Apply LoRa to self-attention layers
        lora_dropout=0.1,
        bias="all",
    )
    lora_model = get_peft_model(base_model, config)
    return accelerator.prepare(lora_model)

In [14]:
from sklearn.metrics import roc_auc_score
def compute_metrics(eval_pred):
    predictions, labels = eval_pred # (B,)
    # print(predictions.shape)
    # predictions = np.argmax(predictions, axis=2)  # Convert logits to class labels
    # labels = accelerator.gather(labels)
    # mask = labels != -100
    # accuracy = (predictions[mask] == labels[mask]).mean()
    accuracy = roc_auc_score(labels, predictions)
    return {'accuracy': accuracy}

In [ ]:
output_dir = f"./training_runs/esm2_t6_8M-finetuned-lora"

args = TrainingArguments(
        output_dir,
        evaluation_strategy="epoch",
        learning_rate=5e-3,
        per_device_train_batch_size=2,
        num_train_epochs=5,
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        save_strategy="epoch",
        label_names=["labels"],
    )

/home/kaustubh/miniforge3/envs/lora_esm/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
trainer = Trainer(
    model=None,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    model_init=model_init,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="wandb",
    hp_space=wandb_hp_space,
    n_trials=10,
)

print("Best Trial:", best_trial)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: pvuaf0c1
Sweep URL: https://wandb.ai/kauamritkar-university-of-wisconsin-madison/uncategorized/sweeps/pvuaf0c1


wandb: Agent Starting Run: uwkevz5s with config:
wandb: 	learning_rate: 0.0008249543621818322
wandb: 	per_device_train_batch_size: 2
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
/home/kaustubh/miniforge3/envs/lora_esm/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


...

wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
/home/kaustubh/miniforge3/envs/lora_esm/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.788992,0.833333
2,0.675900,0.791145,0.916667
3,0.675900,0.798135,1.000000
4,0.682000,0.796367,0.916667
5,0.676700,0.795011,0.916667


/home/kaustubh/miniforge3/envs/lora_esm/lib/python3.13/site-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in /tmp/facebook/esm2_t6_8M_UR50D - will assume that the vocabulary was not modified.
  warnings.warn(
/home/kaustubh/miniforge3/envs/lora_esm/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/kaustubh/miniforge3/envs/lora_esm/lib/python3.13/site-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in /tmp/facebook/esm2_t6_8M_UR50D - will assume that the vocabulary was not modified.
  warnings.warn(
/home/kaustubh/miniforge3/envs/lora_esm/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn

eval/accuracy,▁▅█▅▅
eval/loss,▁▃█▇▆
eval/runtime,▂▁▃▂█
eval/samples_per_second,▇█▆▇▁
eval/steps_per_second,▇█▆▇▁
train/epoch,▁▂▃▅▅▆███
train/global_step,▁▂▃▅▅▆███
train/grad_norm,█▃▁
train/learning_rate,█▅▁
train/loss,▁█▂
eval/accuracy,0.91667


Best Trial: BestRun(run_id='uwkevz5s', objective=1.0, hyperparameters={'learning_rate': 0.0008249543621818322, 'per_device_train_batch_size': 2, 'assignments': {}, 'metric': 'eval/loss'}, run_summary=None)


In [18]:
def train_final_model(best_trial):
    best_hyperparameters = best_trial.hyperparameters
    model = model_init()
    args.learning_rate = best_hyperparameters["learning_rate"]
    args.per_device_train_batch_size = best_hyperparameters["per_device_train_batch_size"]
    final_trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )
    final_trainer.train()
    # Explicitly save the model's configuration
    model.config.save_pretrained(output_dir)
    # Save the model
    final_trainer.save_model(output_dir)
    
    # Log in to Hugging Face account
    # notebook_login()
    # Push the model to the Hub
    # repo_name = "AmelieSchreiber/esm2_t6_8M_UR50D_lora_rna_binding_sites" # Change this to your desired repository name
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    # model.push_to_hub(repo_name)
train_final_model(best_trial)

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
/home/kaustubh/miniforge3/envs/lora_esm/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.695800,0.783984,0.916667
2,0.668800,0.781743,0.916667
3,0.691200,0.800304,0.916667
4,0.672700,0.799796,0.750000
5,0.681300,0.797781,0.750000


/home/kaustubh/miniforge3/envs/lora_esm/lib/python3.13/site-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in /tmp/facebook/esm2_t6_8M_UR50D - will assume that the vocabulary was not modified.
  warnings.warn(
/home/kaustubh/miniforge3/envs/lora_esm/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/kaustubh/miniforge3/envs/lora_esm/lib/python3.13/site-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in /tmp/facebook/esm2_t6_8M_UR50D - will assume that the vocabulary was not modified.
  warnings.warn(
/home/kaustubh/miniforge3/envs/lora_esm/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn

In [ ]:
import itertools

def train_with_hyperparams(learning_rate, batch_size):
    args = TrainingArguments(
        output_dir="./esm2_t6_8M-finetuned-lora",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        num_train_epochs=5,
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
    )

    trainer = Trainer(
        model=model_init(None),
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )
    trainer.train()

    # Evaluate and return accuracy
    results = trainer.evaluate()
    return results["eval_accuracy"]

# Define hyperparameter grid
learning_rates = [1e-5, 5e-5, 1e-4, 5e-4]
batch_sizes = [2, 4, 8, 16]

best_accuracy = 0
best_hyperparams = {}

# Perform grid search
for lr, bs in itertools.product(learning_rates, batch_sizes):
    print(f"Training with learning_rate={lr}, batch_size={bs}")
    accuracy = train_with_hyperparams(lr, bs)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparams = {"learning_rate": lr, "batch_size": bs}

print("Best hyperparameters:", best_hyperparams)